# **--- CNN MODEL ---** #

## **I. Libraries import** ##


In [26]:
import os
import json
from pathlib import Path
import shutil
import zipfile
import requests
from tqdm import tqdm
import random
import mlflow
import tempfile
import boto3

# Torch ------------------
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision import models,datasets
from torchinfo import summary
import torch.nn as nn
import torch.optim as optim

# Metrics ------------------
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix

# Visualization ---------
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image


from dotenv import load_dotenv 

load_dotenv()


True

We select the appropriate torch device

In [27]:
#device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():  # Apple M1/M2/M3
    device = torch.device("mps")
else:
    device = torch.device("cpu")

print(f"Using {device} device")

Using cuda device


## **II. Images import and processing** ##


### Load dataset ###


In [28]:
DATASET_NAME=os.getenv("DATASET_NAME","inrae")
DATASET_DIR=Path(f"../data-{DATASET_NAME}")
print(DATASET_NAME)
print(DATASET_DIR)

inrae
../data-inrae


In [29]:
#  We randomly reduce the amount of images in the "healthy" class that is too represented : 

random.seed(42)

TARGET_NB = 350
sain = Path(f"{DATASET_DIR}/raw_data_inrae/sain")

images = list(sain.glob("*"))
n_images = len(images)

print(f"{sain.name}: {n_images} images")

if n_images <= TARGET_NB:
    print("Nothing to delete")
else:
    images_to_delete = random.sample(
        images, n_images - TARGET_NB
    )

    for img in images_to_delete:
        img.unlink()

    print(f"{len(images_to_delete)} images deleted")

sain: 350 images
Nothing to delete


In [30]:
#  Clean the target folder
organized_data = Path(f"../{DATASET_DIR}/organized_data_inrae")

if organized_data.exists():
    shutil.rmtree(organized_data)


organized_data.mkdir(parents=True, exist_ok=True)

# --------------   Divide the dataset into Train, Val and Test -------------------
random.seed(42)

raw_data_path = Path(f"{DATASET_DIR}/raw_data_inrae")
organized_data = Path(f"{DATASET_DIR}/organized_data_inrae")

SPLITS = {
    "train": 0.7,
    "val": 0.15,
    "test": 0.15
}

# Create folder structure
for split in SPLITS:
    for class_dir in raw_data_path.iterdir():
        if class_dir.is_dir():
            (organized_data / split / class_dir.name).mkdir(parents=True, exist_ok=True)

# Split images
for class_dir in raw_data_path.iterdir():
    if not class_dir.is_dir():
        continue

    images = [
        img for img in class_dir.iterdir()
        if img.suffix.lower() in {".jpg", ".jpeg", ".png"}
    ]

    random.shuffle(images)

    n_total = len(images)
    n_train = int(n_total * SPLITS["train"])
    n_val = int(n_total * SPLITS["val"])

    train_imgs = images[:n_train]
    val_imgs = images[n_train:n_train + n_val]
    test_imgs = images[n_train + n_val:]

    for img in train_imgs:
        shutil.copy(img, organized_data / "train" / class_dir.name / img.name)

    for img in val_imgs:
        shutil.copy(img, organized_data / "val" / class_dir.name / img.name)

    for img in test_imgs:
        shutil.copy(img, organized_data / "test" / class_dir.name / img.name)



In [31]:
# We check the size of the new folders

ROOT_DIR = Path(f"{DATASET_DIR}/organized_data_inrae")

for split in ["train", "val", "test"]:
    print(f"\n{split.upper()}")
    total = 0

    for class_dir in (ROOT_DIR / split).iterdir():
        if class_dir.is_dir():
            n_files = len(list(class_dir.glob("*")))
            total += n_files
            print(f"  {class_dir.name:<20} : {n_files}")

    print(f"  TOTAL {split:<14} : {total}")




TRAIN
  colomerus_vitis      : 119
  erysiphe_necator     : 91
  guignardia_bidwellii : 169
  phaeomoniella_chlamydospora : 82
  plasmopara_viticola  : 345
  sain                 : 318
  elsinoe_ampelina     : 262
  TOTAL train          : 1386

VAL
  colomerus_vitis      : 34
  erysiphe_necator     : 27
  guignardia_bidwellii : 51
  phaeomoniella_chlamydospora : 25
  plasmopara_viticola  : 106
  sain                 : 98
  elsinoe_ampelina     : 81
  TOTAL val            : 422

TEST
  colomerus_vitis      : 38
  erysiphe_necator     : 31
  guignardia_bidwellii : 54
  phaeomoniella_chlamydospora : 25
  plasmopara_viticola  : 105
  sain                 : 98
  elsinoe_ampelina     : 83
  TOTAL test           : 434


### Pipeline for data transformation ###


In [32]:
# Transformations of the train set with data augmentation

transform_train= transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(degrees=(-45,+45)),
    transforms.ToTensor(),        
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225])  # Normalization with the values for the pre-trained Resnet model
])

In [33]:
# Create the path to the training dataset
data_train = Path(f"{DATASET_DIR}/organized_data_inrae/train")

# Load dataset with ImageFolder
train_dataset = ImageFolder(root=data_train, transform=transform_train)

# Get class names
class_names = train_dataset.classes

We build a dictionnary with translations and we send a JSON file to our S3 bucket

In [34]:
translations = [
    "Erinose",
    "Anthracnose",
    "Oïdium",
    "Pourriture_noire",
    "Esca",
    "Mildiou",
    "Pas de maladie"
]

DISEASES = {}
for c,t in zip(class_names, translations):
    DISEASES[c] = t

print(json.dumps(DISEASES, indent=4, ensure_ascii=False))

S3_BUCKET_NAME= os.getenv('S3_BUCKET_NAME', "aws-s3-mlflow")

# Temporary file for saving the confusion matrices
with tempfile.TemporaryDirectory() as tmp_dir:
        s3 = boto3.client('s3')
        path = Path(tmp_dir, f"disease-{DATASET_NAME}.json")
        print(str(path))
        with path.open('w') as f:
            json.dump(DISEASES, f)
        dest_file_name = f'vitiscan-data/disease-{DATASET_NAME}.json'
        s3.upload_file(Bucket=S3_BUCKET_NAME, Filename=str(path), Key=dest_file_name)
        s3.close()
        print(f"Disease dictionnary uploaded to : s3://{S3_BUCKET_NAME}/{dest_file_name}")

{
    "colomerus_vitis": "Erinose",
    "elsinoe_ampelina": "Anthracnose",
    "erysiphe_necator": "Oïdium",
    "guignardia_bidwellii": "Pourriture_noire",
    "phaeomoniella_chlamydospora": "Esca",
    "plasmopara_viticola": "Mildiou",
    "sain": "Pas de maladie"
}
/tmp/tmpb4f_pase/disease-inrae.json
Disease dictionnary uploaded to : s3://aws-s3-mlflow/vitiscan-data/disease-inrae.json


In [35]:
#  Transformation pipeline without data augmentation for the validation and the test set

transform= transforms.Compose([
    transforms.Resize((224, 224)),  # Redimensionnement à 224x224
    transforms.ToTensor(),          # Conversion en tenseur
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225])  # Normalisation avec les valeurs du modèle pré-entraîné Resnet
])

# Create the path to the training dataset
data_val = Path(f"{DATASET_DIR}/organized_data_inrae/val")
data_test = Path(f"{DATASET_DIR}/organized_data_inrae/test")

# Load dataset with ImageFolder
val_dataset = ImageFolder(root=data_val,transform=transform)
test_dataset = ImageFolder(root=data_test, transform=transform)


In [36]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

## **Fine tuning (Resnet18)** ##

### Preparing the MLFlow tracking ###

In [37]:
# Set tracking URI to your Hugging Face application
MLFLOW_URI=os.getenv('MLFLOW_URI',"https://gviel-mlflow37.hf.space/")
mlflow.set_tracking_uri(os.environ["MLFLOW_URI"])

# Set experiment's info
EXPERIMENT_NAME= os.getenv('EXPERIMENT_NAME',"Vitiscan_CNN_MLFlow_FineTuning")+"FINE_TUNING"
mlflow.set_experiment(EXPERIMENT_NAME)

# Get our experiment info
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)

### Importing a pre-trained model

In [38]:
#model = models.resnet18(pretrained=True) # pour version <0.13.0
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT) # pour version >=0.13.0
model = model.to(device)
print(f"Model device: {next(model.parameters()).device}")
print(device)

Model device: cuda:0
cuda


### Replace classification layer to adapt the model to our features ###

In [39]:
nb_classes=len(train_dataset.classes)
model.fc = nn.Linear(model.fc.in_features, nb_classes)


### Freeze the feature extraction layers ###

In [40]:
# We freeze the entire network
for param in model.parameters():
    param.requires_grad = False

# We unfreeze only the last Resnet18 blocks of layers (called "Layer4 for resnet18")
# AND we unfreeze the classifier fc
for name, param in model.named_parameters():
    if "layer4" in name or "fc" in name:
        param.requires_grad = True

In [41]:
# Print model summary
summary(model, input_size=(1, 3, 224, 224))  # (batch_size, input_features)

Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   [1, 7]                    --
├─Conv2d: 1-1                            [1, 64, 112, 112]         (9,408)
├─BatchNorm2d: 1-2                       [1, 64, 112, 112]         (128)
├─ReLU: 1-3                              [1, 64, 112, 112]         --
├─MaxPool2d: 1-4                         [1, 64, 56, 56]           --
├─Sequential: 1-5                        [1, 64, 56, 56]           --
│    └─BasicBlock: 2-1                   [1, 64, 56, 56]           --
│    │    └─Conv2d: 3-1                  [1, 64, 56, 56]           (36,864)
│    │    └─BatchNorm2d: 3-2             [1, 64, 56, 56]           (128)
│    │    └─ReLU: 3-3                    [1, 64, 56, 56]           --
│    │    └─Conv2d: 3-4                  [1, 64, 56, 56]           (36,864)
│    │    └─BatchNorm2d: 3-5             [1, 64, 56, 56]           (128)
│    │    └─ReLU: 3-6                    [1, 64, 56, 56]   

### Defining the cost function and optimizer ###

In [42]:
criterion = nn.CrossEntropyLoss()
criterion = criterion.to(device) 

learning_rate=0.0001

optimizer = torch.optim.Adam(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=learning_rate,
    weight_decay=0.0001 # Ridge regulation to avoid overfitting
)

### Train the model ###

We check an output on the fist batch

In [43]:
# Model outputs
image, label = next(iter(train_loader))
image=image.to(device)
model=model.to(device)
logit= model(image) # Resnet18 output
logit
# Attention il faut que l'image et le modele aient le même device

tensor([[-0.2581,  0.0351, -0.1942, -0.3909, -0.1313, -0.6100, -0.8514],
        [-0.0844,  0.7455, -0.1052, -0.3020,  0.0630, -0.4493, -0.6740],
        [ 0.8736,  0.0063,  0.0579, -0.0556, -0.0840, -0.3114, -0.7350],
        [ 0.5816,  0.9475, -0.3270,  0.7279,  0.5075, -0.1729, -1.0160],
        [ 0.4482,  0.6876,  0.1040, -0.4828, -0.1668, -1.2552, -0.0777],
        [ 0.9259,  0.8473,  0.2512, -0.5780, -0.6492, -0.4590, -0.4265],
        [ 0.9657,  0.7889,  0.4453, -0.2684,  0.6325, -0.7888, -0.2608],
        [ 0.6828,  0.5779, -0.1858, -0.1814,  0.3079, -0.4639, -0.8463],
        [ 0.1773,  0.2449, -0.3121,  0.0227,  0.6181, -0.6690, -0.9461],
        [ 0.4295,  0.7947, -0.3575, -0.2658, -0.0257, -0.4239, -0.1446],
        [ 1.3525,  0.6106, -0.1213, -0.3091,  0.7801, -0.4456,  0.1556],
        [ 1.0808,  1.1653, -0.0967,  0.2634,  0.3628, -0.9215, -1.5761],
        [-0.0271,  0.1723, -0.0329, -0.8453,  0.5059, -0.6402, -0.5270],
        [ 0.4036,  0.5703, -0.1502,  0.8694, -0.344

In [44]:
class_names

['colomerus_vitis',
 'elsinoe_ampelina',
 'erysiphe_necator',
 'guignardia_bidwellii',
 'phaeomoniella_chlamydospora',
 'plasmopara_viticola',
 'sain']

In [45]:
def evaluate_model_on_test(model, test_loader, device):
    
    model.eval() # Mode évaluation
    y_true = []
    y_pred = []
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            logit = model(images)
            preds = logit.argmax(dim=1)
            
            # Compter l'exactitude
            total += labels.size(0)
            correct += (preds == labels).sum().item()
            
            # Collecter pour les métriques de Scikit-learn
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
    
    test_accuracy = correct / total
    return test_accuracy, y_true, y_pred

In [46]:
def log_confusion_matrix(dataset_type:str, y_true:list, y_pred:list):
    ''' Generate a Confusion Matrix and log it into MLFlow'''
    with tempfile.TemporaryDirectory() as tmp_dir:
            cm_test = confusion_matrix(y_true, y_pred)
            plt.figure(figsize=(10, 8))
            sns.heatmap(cm_test, annot=True, fmt='d', cmap='Blues')
            plt.xticks(ticks= range(nb_classes),labels=class_names, rotation=45, ha="right")
            plt.yticks(ticks= range(nb_classes),labels=class_names,rotation=0)
            plt.ylabel('True class')
            plt.xlabel('Predicted class')
            plt.title(f'Confusion matrix - {dataset_type}')
            plt.tight_layout()
            path = str(Path(tmp_dir, f"confusion_matrix_{dataset_type}.png"))
            plt.savefig(path, dpi=150)
            mlflow.log_artifact(path)
            plt.close()

In [ ]:
def log_precision_recall_f1_score(dataset_type:str, y_true:list, y_pred:list):
    '''
        Compute various scores and log them into MLFlow
        Return the scores in a dict
    '''
    results = {}
    average_modes=['weighted', 'macro']
    for avg_mode in average_modes:
        results[avg_mode] = {}
        for score in ['precision', 'recall', 'f1']:
            # --- Precision ---
            if score == 'precision':
                metric_value = precision_score(y_true, y_pred, average=avg_mode, zero_division=0)
            elif score == 'recall':
                metric_value = recall_score(y_true, y_pred, average=avg_mode, zero_division=0)
            elif score == 'f1':
                metric_value = f1_score(y_true, y_pred, average=avg_mode, zero_division=0)
            
            metric_name = f"{dataset_type.capitalize()}_{score}_{avg_mode}"
            results[avg_mode][score] = metric_value
            mlflow.log_metric(metric_name, metric_value)
            
    return results

In [52]:
# Training function for a PyTorch model
def train(model, train_loader, val_loader, test_loader, criterion, optimizer, experiment, epochs=20, patience=5):
    
    # Early stopping variables
    best_val_loss = float('inf')
    epochs_no_improve = 0
    best_model_state = None

    # We start a MLflow run
    with mlflow.start_run(experiment_id=experiment.experiment_id):

        # Logging Pytorch parameters into MLflow 
        params = {
            "optimizer": type(optimizer).__name__,
            "learning_rate": optimizer.param_groups[0]['lr'],
            "epochs": epochs,
            "criterion": type(criterion).__name__,
            "model_architecture": type(model).__name__,
            "training_device": str(device),
            "weight_decay": optimizer.param_groups[0]["weight_decay"]
        }
        
        mlflow.log_params(params=params)
        mlflow.pytorch.autolog()

        # Dictionary to store loss and accuracy values for each epoch
        history = {
            'loss': [],
            'val_loss': [],
            'accuracy': [],
            'val_accuracy': []
        }

        # ------------------- TRAINING LOOP -------------------
        for epoch in range(epochs):
            model.train()
            total_loss, correct = 0, 0  

            for images, labels in train_loader:
                images, labels = images.to(device), labels.to(device)
                optimizer.zero_grad()
                logit = model(images)
                loss = criterion(logit, labels)
                loss.backward()
                optimizer.step()

                total_loss += loss.item()
                correct += (logit.argmax(dim=1) == labels).sum().item()

            train_loss = total_loss / len(train_loader)
            train_acc = correct / len(train_loader.dataset)

            # ------------------- VALIDATION LOOP -------------------
            model.eval()
            val_loss, val_correct = 0, 0

            with torch.no_grad():
                for images, labels in val_loader:
                    images, labels = images.to(device), labels.to(device)
                    logit = model(images)
                    loss = criterion(logit, labels)
                    val_loss += loss.item()
                    val_correct += (logit.argmax(dim=1) == labels).sum().item()

            val_loss /= len(val_loader)
            val_acc = val_correct / len(val_loader.dataset)

            # --- Save metrics ---
            history['loss'].append(train_loss)
            history['val_loss'].append(val_loss)
            history['accuracy'].append(train_acc)
            history['val_accuracy'].append(val_acc)

            print(
                f"Epoch [{epoch+1}/{epochs}], "
                f"Loss: {train_loss:.4f}, Acc: {train_acc:.4f}, "
                f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}"
            )

            #  Logging metrics
            mlflow.log_metric("train_loss", train_loss, step=epoch)
            mlflow.log_metric("train_accuracy", train_acc, step=epoch)
            mlflow.log_metric("validation_loss", val_loss, step=epoch)
            mlflow.log_metric("validation_accuracy", val_acc, step=epoch)

            # --- Early stopping check ---
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                epochs_no_improve = 0
                best_model_state = model.state_dict()  # save the best model
            else:
                epochs_no_improve += 1
                if epochs_no_improve >= patience:
                    print(f"Early stopping triggered after {epoch+1} epochs")
                    model.load_state_dict(best_model_state)  # restore best model
                    break
                

        # ================= FINAL VALIDATION EVALUATION =================
        print("\n--- Final evaluation on the VALIDATION set ---\n")

        y_true_val, y_pred_val = [], []
        model.eval()

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                logit = model(images)
                preds = logit.argmax(dim=1)

                y_true_val.extend(labels.cpu().numpy())
                y_pred_val.extend(preds.cpu().numpy())

        # --- Compute scores (Validation) ---
        val_scores = log_precision_recall_f1_score('validation', y_true_val, y_pred_val)
        for mode in val_scores.keys():
            for score in val_scores[mode].keys():
                print(f"{score.capitalize()} Validation ({mode}): {val_scores[mode][score]:.4f}")
                
        # Final metrics logging
        mlflow.log_metric("final_validation_accuracy", val_acc)
        print(f"Final Validation Accuracy: {val_acc:.4f}")

        # --- Confusion Matrix (Validation) ---
        log_confusion_matrix('VALIDATION', y_true_val, y_pred_val)

        # ================= FINAL TEST EVALUATION =================
        print("\n--- Final evaluation on the TEST set ---\n")
        test_acc, y_true_test, y_pred_test = evaluate_model_on_test(model, test_loader, device)

        # --- Compute scores (test) ---
        test_scores = log_precision_recall_f1_score('test', y_true_test, y_pred_test)
        for mode in test_scores.keys():
            for score in test_scores[mode].keys():
                print(f"{score.capitalize()} Validation ({mode}): {test_scores[mode][score]:.4f}")

        # --- Accuracy (test) ---
        mlflow.log_metric("Test_accuracy", test_acc)
        print(f"Test Accuracy: {test_acc:.4f}")

        # --- Confusion Matrix (Test) ---
        log_confusion_matrix('TEST', y_true_test, y_pred_test)

        # --- Model logging ---
        mlflow.pytorch.log_model(
            pytorch_model=model,
            artifact_path="Resnet18",
            registered_model_name=f"{params['model_architecture']}"
        )

        print("\n--- Metrics and model logged into MLflow ---\n")

        return history


In [49]:
'''
DISEASES = {
    "colomerus_vitis" : "erinose",
    "elsinoe_ampelina" : "anthracnose",
    "erysiphe_necator":"oidium",
    "guignardia_bidwellii" : "pourriture_noire",
    "phaeomoniella_chlamydospora" : "esca",
    "plasmopara_viticola":"mildiou",
    "sain" : "sain"
    }
'''
print(json.dumps(DISEASES, indent=4, ensure_ascii=False))


{
    "colomerus_vitis": "Erinose",
    "elsinoe_ampelina": "Anthracnose",
    "erysiphe_necator": "Oïdium",
    "guignardia_bidwellii": "Pourriture_noire",
    "phaeomoniella_chlamydospora": "Esca",
    "plasmopara_viticola": "Mildiou",
    "sain": "Pas de maladie"
}


In [53]:
# Train the model and store the training history
history = train(model, train_loader, val_loader, test_loader, criterion, optimizer, experiment, epochs=3, patience=5)


Epoch [1/3], Loss: 0.2445, Acc: 0.9192, Val Loss: 0.1728, Val Acc: 0.9455
Epoch [2/3], Loss: 0.1852, Acc: 0.9488, Val Loss: 0.1803, Val Acc: 0.9384
Epoch [3/3], Loss: 0.1654, Acc: 0.9466, Val Loss: 0.1388, Val Acc: 0.9526

--- Final evaluation on the VALIDATION set ---

Precision Validation (weighted): 0.9571
Recall Validation (weighted): 0.9526
F1 Validation (weighted): 0.9511
Precision Validation (macro): 0.9630
Recall Validation (macro): 0.9365
F1 Validation (macro): 0.9454
Final Validation Accuracy: 0.9526

--- Final evaluation on the TEST set ---

Precision Validation (weighted): 0.9542
Recall Validation (weighted): 0.9493
F1 Validation (weighted): 0.9476
Precision Validation (macro): 0.9634
Recall Validation (macro): 0.9406
F1 Validation (macro): 0.9476
Test Accuracy: 0.9493


2025/12/19 19:48:30 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
Registered model 'ResNet' already exists. Creating a new version of this model...
2025/12/19 19:49:01 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ResNet, version 24
Created version '24' of model 'ResNet'.



--- Metrics and model logged into MLflow ---

🏃 View run debonair-crab-123 at: https://gviel-mlflow37.hf.space/#/experiments/5/runs/2cdb6b01fb8b4355870cfe2d05697eaf
🧪 View experiment at: https://gviel-mlflow37.hf.space/#/experiments/5


### Visualization of the learning process ###

In [ ]:
from plotly import graph_objects as go
color_chart = ["#4B9AC7", "#4BE8E0", "#9DD4F3", "#97FBF6", "#2A7FAF", "#23B1AB", "#0E3449", "#015955"]

fig = go.Figure(data=[
                      go.Scatter(
                          y=history["loss"],
                          name="Training loss",
                          mode="lines",
                          marker=dict(
                              color=color_chart[0]
                          )),
                      go.Scatter(
                          y=history["val_loss"],
                          name="Validation loss",
                          mode="lines",
                          marker=dict(
                              color=color_chart[1]
                          ))
])
fig.update_layout(
    title='Training and val loss across epochs',
    xaxis_title='epochs',
    yaxis_title='Cross Entropy'
)
fig.show()

: 

In [ ]:
color_chart = ["#4B9AC7", "#4BE8E0", "#9DD4F3", "#97FBF6", "#2A7FAF", "#23B1AB", "#0E3449", "#015955"]

fig = go.Figure(data=[
                      go.Scatter(
                          y=history["accuracy"],
                          name="Training Accuracy",
                          mode="lines",
                          marker=dict(
                              color=color_chart[0]
                          )),
                      go.Scatter(
                          y=history["val_accuracy"],
                          name="Validation Accuracy",
                          mode="lines",
                          marker=dict(
                              color=color_chart[1]
                          ))
])
fig.update_layout(
    title='Training and val Accuracy across epochs',
    xaxis_title='epochs',
    yaxis_title='Cross Entropy'
)
fig.show()

: 